# 神经网络模型：

In [301]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 100

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/sk_space_1d_rate_20d_up_train'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = data_path[5:7]+'_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1 + '_up'

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

sk_c2_s0.1_b16_lr0.001_d0.5_e100_up
models/model_sk_c2_s0.1_b16_lr0.001_d0.5_e100_up.pth
True
1
GeForce MX150
0


### 构建训练集

In [302]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,0.0,600376.XSHG,MainBoard,RealEstate,0,2014-09-01,0.155301,1.389569,-0.742512,1.004896,...,-0.723738,-0.340049,-0.606578,-0.066340,-0.210841,0.083270,-1.045114,-0.393679,-0.872230,-0.719400
1,0.0,600376.XSHG,MainBoard,RealEstate,1,2014-09-02,0.199428,1.357446,-0.724225,0.971029,...,-0.103967,0.426934,-0.429731,-0.086189,-0.211403,0.066068,-1.074718,-0.392116,-0.807516,-0.830423
2,0.0,600376.XSHG,MainBoard,RealEstate,2,2014-09-03,0.046038,1.319701,-0.702279,0.931236,...,0.421506,0.426934,-0.246464,-0.087480,-0.202554,0.054438,-1.073104,-0.389364,-0.601557,-0.647419
3,0.0,600376.XSHG,MainBoard,RealEstate,3,2014-09-04,0.314559,1.282805,-0.680334,0.892337,...,0.368239,0.426934,-0.082092,-0.157214,-0.193872,0.052077,-1.054760,-0.387450,-0.132819,-0.188955
4,0.0,600376.XSHG,MainBoard,RealEstate,4,2014-09-05,0.495197,1.295012,-0.687649,0.905206,...,0.155170,0.426934,-0.160803,-0.182925,-0.189293,0.048252,-1.045388,-0.387953,-0.451531,-0.477728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,1.0,600376.XSHG,MainBoard,RealEstate,522,2019-03-26,-0.447526,1.101898,-0.840072,1.256462,...,1.215658,-1.107033,0.357106,-0.575565,-0.695986,-0.803012,1.098151,0.541113,1.196026,2.311781
508,1.0,600376.XSHG,MainBoard,RealEstate,523,2019-03-27,-0.664240,1.108614,-0.843992,1.264758,...,0.163493,-1.874017,0.212989,-0.579929,-0.694342,-0.802500,1.036168,0.577849,0.955672,2.051009
509,1.0,600376.XSHG,MainBoard,RealEstate,524,2019-03-28,-0.429381,1.195120,-0.892992,1.371614,...,-1.154517,-2.641001,-0.324107,-0.578545,-0.692693,-0.801382,0.942849,0.593940,0.750872,1.797271
510,1.0,600376.XSHG,MainBoard,RealEstate,525,2019-03-29,-1.031287,0.973481,-0.761673,1.097834,...,0.088586,-2.641001,0.752425,-0.569861,-0.686683,-0.798262,0.916414,0.647282,1.503490,2.550063


In [303]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
79,0.310367,0.438979,-0.294945,0.751335,0.864112,-0.125413,1.024170,0.426934,1.671853,1.382365,0.886554,0.667063,-0.097866,0.581123,0.629129,0.987442
382,-0.961220,-0.413666,-0.442000,-0.546780,-0.615578,0.016755,-0.026086,0.426934,-0.261236,-0.781431,-0.793854,-0.531507,-1.008564,-0.947869,-0.664390,-0.409365
198,-0.704593,0.097075,-0.533676,-0.507531,-0.097829,-1.716827,-1.755353,-0.340049,-1.113178,-0.382990,-0.441483,-0.384283,0.392090,-0.222195,2.283511,0.601296
417,-0.625312,-0.548998,-0.393653,-0.890804,-0.890359,-0.374639,-1.877524,-1.107033,-0.798939,-0.824048,-0.826394,-0.825700,-0.704042,-0.832161,-1.224591,-0.942376
493,1.407756,-1.002750,1.634275,0.381003,0.177336,0.882659,1.772208,2.727886,0.950798,-0.727155,-0.778790,-0.864582,0.129387,-0.414433,0.861440,1.890209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,-0.813878,3.291659,-1.310116,3.360619,2.969722,2.764531,0.978601,0.426934,2.042504,2.269002,2.231730,2.679046,3.030261,3.072342,0.278871,1.060516
112,-0.026166,0.972536,-0.625767,-0.205537,0.219807,-1.652657,-0.571046,-1.107033,-0.859580,1.577855,2.007490,1.362873,0.801238,1.849816,0.417637,-0.246091
173,-0.716752,2.830277,-1.207039,2.320741,2.284990,1.048269,1.315469,0.426934,1.625157,2.138479,2.075666,2.650899,1.916798,2.410884,0.046822,0.869918
220,-0.656152,-0.004056,-0.456864,-0.971758,-1.054307,-0.103754,-0.836701,-0.340049,-0.495085,-0.715596,-0.553193,-0.437514,-0.795005,-0.175838,-1.247266,-1.362879


In [304]:
y_train

79     0.0
382    1.0
198    1.0
417    1.0
493    0.0
      ... 
177    1.0
112    0.0
173    1.0
220    0.0
381    1.0
Name: return_bin, Length: 460, dtype: float64

In [305]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
79,0.310367,0.438979,-0.294945,0.751335,0.864112,-0.125413,1.024170,0.426934,1.671853,1.382365,0.886554,0.667063,-0.097866,0.581123,0.629129,0.987442,0.0
382,-0.961220,-0.413666,-0.442000,-0.546780,-0.615578,0.016755,-0.026086,0.426934,-0.261236,-0.781431,-0.793854,-0.531507,-1.008564,-0.947869,-0.664390,-0.409365,1.0
198,-0.704593,0.097075,-0.533676,-0.507531,-0.097829,-1.716827,-1.755353,-0.340049,-1.113178,-0.382990,-0.441483,-0.384283,0.392090,-0.222195,2.283511,0.601296,1.0
417,-0.625312,-0.548998,-0.393653,-0.890804,-0.890359,-0.374639,-1.877524,-1.107033,-0.798939,-0.824048,-0.826394,-0.825700,-0.704042,-0.832161,-1.224591,-0.942376,1.0
493,1.407756,-1.002750,1.634275,0.381003,0.177336,0.882659,1.772208,2.727886,0.950798,-0.727155,-0.778790,-0.864582,0.129387,-0.414433,0.861440,1.890209,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,-0.813878,3.291659,-1.310116,3.360619,2.969722,2.764531,0.978601,0.426934,2.042504,2.269002,2.231730,2.679046,3.030261,3.072342,0.278871,1.060516,1.0
112,-0.026166,0.972536,-0.625767,-0.205537,0.219807,-1.652657,-0.571046,-1.107033,-0.859580,1.577855,2.007490,1.362873,0.801238,1.849816,0.417637,-0.246091,0.0
173,-0.716752,2.830277,-1.207039,2.320741,2.284990,1.048269,1.315469,0.426934,1.625157,2.138479,2.075666,2.650899,1.916798,2.410884,0.046822,0.869918,1.0
220,-0.656152,-0.004056,-0.456864,-0.971758,-1.054307,-0.103754,-0.836701,-0.340049,-0.495085,-0.715596,-0.553193,-0.437514,-0.795005,-0.175838,-1.247266,-1.362879,0.0


In [306]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [307]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [308]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [309]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [310]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [311]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [312]:
from my_utils.model_class import MLP

model = MLP(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [313]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [314]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [315]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [316]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 53.7%, Avg loss: 0.689716 

Precision of every train dataset class:  tensor([0.3907, 0.6084], device='cuda:0')

Test Error: 
    Accuracy: 51.9%, Avg loss: 0.685483 

Precision of every test dataset class:  tensor([0.3750, 0.5455], device='cuda:0')

Time cost = 0.343082s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 59.1%, Avg loss: 0.678976 

Precision of every train dataset class:  tensor([0.4474, 0.6198], device='cuda:0')

Test Error: 
    Accuracy: 53.8%, Avg loss: 0.673531 

Precision of every test dataset class:  tensor([0.4000, 0.5532], device='cuda:0')

Time cost = 0.709515s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 64.1%, Avg loss: 0.664971 

Precision of every train dataset class:  tensor([0.6119, 0.6463], device='cuda:0')

Test Error: 
    Accuracy: 51.9%, Avg loss: 0.672375 

Precision of every test dataset class:  tensor([0.2500, 0.5417], device='cuda:0')



## 保存模型

In [317]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [318]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [319]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [320]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [321]:
# X_cv.columns.values.tolist()

In [322]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [323]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [324]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)